In [1]:
from some_functions import *

# Задание 1

### Подготовка исходных данных

In [2]:
# Get data
import pandas as pd
df = pd.read_excel(r'data\Train.xlsx')

In [3]:
train_data=get_monthly_data_prepared(df)

In [4]:
X_train,y_train=x_y_split_task1(train_data)

### Обучение лучшей модели

In [5]:
# fit best model
import xgboost as xgb
xgb_reg = xgb.XGBRegressor(base_score=0.5, booster='gbtree',    
                       n_estimators=2400,
                       early_stopping_rounds=50,
                       objective='reg:linear',
                       max_depth=7,
                       learning_rate=0.007,
                          reg_lambda=0.9)

xgb_reg.fit(X_train, y_train)
print ("WAPE SCORE ON TRAIN IS ", get_score(xgb_reg.predict(X_train),y_train))

[21:49:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[21:49:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "early_stopping_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


WAPE SCORE ON TRAIN IS  0.008102808233119237


### Подготовка данных для предсказания

In [6]:
import dateutil.relativedelta
# Data to predict
next_month = df['MON_DATE'].max()+dateutil.relativedelta.relativedelta(months=1)
all_stores_id = df['Store'].unique()
next_month,all_stores_id

(Timestamp('2015-07-01 00:00:00'),
 array([  25,   94,  116,  155,  174,  192,  325,  349,  376,  394,  417,
         438,  473,  484,  536,  601,  629,  757,  767,  795,  807,  824,
         841,  934,  941,  942, 1045, 1081], dtype=int64))

In [7]:
X_predict = pd.DataFrame(all_stores_id,columns=["Store"])
X_predict['MON_DATE']=next_month
X_predict=X_predict.set_index('MON_DATE')
X_predict=create_features_v1(X_predict)

### Предсказание

In [8]:
predictions = xgb_reg.predict(X_predict)
df_predicted=pd.DataFrame()
df_predicted['Store']=X_predict['Store']
df_predicted['Predicted sales']=predictions
df_predicted=df_predicted.astype({'Predicted sales':int})

### Результат

In [9]:
# save to excel
df_predicted.to_excel(r'Predictions\Task1_next_month.xlsx')

# Задание 2

### Подготовка исходных данных

In [10]:
train_data_2=get_daily_data_prepared(df)

In [11]:
X_train_2,y_train_2=x_y_split_task2(train_data_2)

In [12]:
X_train_2

,Store,dayofyear,dayofmonth,dayofweek,year,quarter,month,has_major_holiday_coming,weekofyear
Date,,,,,,,,,
2015-06-30,25,181,30,1,2015,2,6,0,27
2015-06-30,94,181,30,1,2015,2,6,0,27
2015-06-30,116,181,30,1,2015,2,6,0,27
2015-06-30,155,181,30,1,2015,2,6,0,27
2015-06-30,174,181,30,1,2015,2,6,0,27
...,...,...,...,...,...,...,...,...,...
2013-01-02,841,2,2,2,2013,1,1,0,1
2013-01-02,934,2,2,2,2013,1,1,0,1
2013-01-02,941,2,2,2,2013,1,1,0,1


### Обучение лучшей модели

In [13]:
# fit best model
xgb_reg_2 = xgb.XGBRegressor(base_score=0.5, booster='gbtree',    
                       n_estimators=3200,
                       early_stopping_rounds=100,
                       objective='reg:linear',
                       max_depth=8,
                       learning_rate=0.005)
xgb_reg_2.fit(X_train_2, y_train_2)
print ("WAPE SCORE ON TRAIN IS ", get_score(xgb_reg_2.predict(X_train_2),y_train_2))

[21:49:06] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[21:49:06] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "early_stopping_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


WAPE SCORE ON TRAIN IS  0.06349554429566943


### Подготовка данных для предсказания

In [14]:
import numpy as np
start_day = max(df['Date'])+ np.timedelta64(1, 'D')
end_day = start_day + np.timedelta64(3, 'M')
start_day,end_day

(Timestamp('2015-07-01 00:00:00'), Timestamp('2015-09-30 07:27:18'))

In [15]:
all_days=pd.date_range(start_day,end_day,freq='d')

In [16]:
X_predict_2 = pd.DataFrame(all_days,columns=["Date"])
X_predict_2["tmp"] =1
all_stores_df = pd.DataFrame(all_stores_id,columns=["Store"])
all_stores_df["tmp"] =1
X_predict_2=X_predict_2.merge(all_stores_df, how='outer')
X_predict_2=X_predict_2.drop(columns=["tmp"])
X_predict_2=X_predict_2.set_index('Date')
X_predict_2=create_features_v1(X_predict_2)
X_predict_2

,Store,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,has_major_holiday_coming
Date,,,,,,,,,
2015-07-01,25,2,3,7,2015,182,1,27,0
2015-07-01,94,2,3,7,2015,182,1,27,0
2015-07-01,116,2,3,7,2015,182,1,27,0
2015-07-01,155,2,3,7,2015,182,1,27,0
2015-07-01,174,2,3,7,2015,182,1,27,0
...,...,...,...,...,...,...,...,...,...
2015-09-30,934,2,3,9,2015,273,30,40,0
2015-09-30,941,2,3,9,2015,273,30,40,0
2015-09-30,942,2,3,9,2015,273,30,40,0


### Предсказание

In [17]:
predictions_2 = xgb_reg_2.predict(X_predict_2)
df_predicted_2=pd.DataFrame()
df_predicted_2['Store']=X_predict_2['Store']
df_predicted_2['Predicted sales']=predictions_2
df_predicted_2=df_predicted_2.astype({'Predicted sales':int})
df_predicted_2

,Store,Predicted sales
Date,,
2015-07-01,25,9675
2015-07-01,94,5520
2015-07-01,116,6045
2015-07-01,155,6471
2015-07-01,174,6456
...,...,...
2015-09-30,934,3943
2015-09-30,941,3318
2015-09-30,942,3108


### Результат

In [18]:
df_predicted_2.to_excel(r'Predictions\Task2_next_3_months.xlsx')